# Homework 5

Spencer Pease  
ASTR 427

----------------------------

In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rand

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

%matplotlib inline

## Problem 1

In [2]:
# estimate pi with cpu

def monteCarlo(n):
    
    # number of guesses that fall on or within the circle
    inside = 0
    
    for _ in range(n):
        x = rand.random()
        y = rand.random()
        
        if x**2 + y**2 <= 1:
            inside += 1
    
    # area for one quadrant of the circle
    area = float(inside)/n
    
    # area of the entire circle (~pi)
    return area*4

print "pi:", monteCarlo(100000)

pi: 3.1586


In [3]:
# plotting error



## Problem 2

In [4]:
# estimate pi with gpu

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0